In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import cv2
import os
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image
from torchvision import transforms, models

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 384)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])

In [ ]:
def load_image(image_path, title):
    image = Image.open(image_path)
    plt.figure()
    plt.imshow(image)
    plt.title(title)
    plt.show()
    image = transform(image).unsqueeze(0)
    return image.to(device)

In [ ]:
content_image_path = "/kaggle/input/wonders-of-the-world-image-classification/Wonders of World/Wonders of World/eiffel_tower/01de31a300.jpg"
content_image = load_image(content_image_path, "Content Image")

In [ ]:
style_image_path = "/kaggle/input/best-artworks-of-all-time/images/images/Vincent_van_Gogh/Vincent_van_Gogh_100.jpg"
style_image = load_image(style_image_path, "Style Image")

# Model

In [ ]:
def get_features(x, model, layers):
    features = {}
    for name, layer in enumerate(model.children()):
        x = layer(x)
        if str(name) in layers:
            features[layers[str(name)]] = x
    return features

In [ ]:
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

In [ ]:
vgg = models.vgg19(weights=True).features.to(device).eval()

In [ ]:
feature_layers = {}
layer_count = 0

for name, layer in vgg._modules.items():
    if isinstance(layer, torch.nn.Conv2d):
        layer_name = f'conv{layer_count//4 + 1}_{layer_count%4 + 1}'
        feature_layers[name] = layer_name
        layer_count += 1

content_layer = 'conv4_2'
num_style_layers = len(feature_layers) - 1
style_layers_dict = {layer: 1.0 / num_style_layers for layer in feature_layers.values()}

In [ ]:
content_features = get_features(content_image, vgg, feature_layers)
style_features = get_features(style_image, vgg, feature_layers)

In [ ]:
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

In [ ]:
target = content_image.clone().requires_grad_(True).to(device)

# Train

In [ ]:
style_weight = 1e6
content_weight = 1
optimizer = optim.Adam([target], lr=0.003)
steps = 2000

In [ ]:
style_losses = []
content_losses = []

In [ ]:
for epoch in range(steps + 1):
    optimizer.zero_grad()
    
    target_features = get_features(target, vgg, feature_layers)
    
    content_loss = F.mse_loss(target_features[content_layer], content_features[content_layer])
    content_losses.append(content_loss.item())
    
    style_loss = 0
    for layer in style_layers_dict:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_style_loss = style_layers_dict[layer] * F.mse_loss(target_gram, style_gram)
        style_loss += layer_style_loss / (target_feature.shape[1] * target_feature.shape[2] * target_feature.shape[3])

    style_losses.append(style_loss.item())
    
    neural_loss = content_weight * content_loss + style_weight * style_loss
    neural_loss.backward(retain_graph=True)
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{steps}], Content Loss: {content_loss.item():.2}, Style Loss {style_loss.item():.2}')

# Results

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(content_losses)
plt.plot(style_losses)
plt.title("Loss Curve")
plt.legend(["Content Loss", "Style Loss"])
plt.show()

In [ ]:
def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image = image * (0.229, 0.224, 0.225) + (0.485, 0.456, 0.406)
    image = image.clip(0, 1)
    return image

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(im_convert(target))
plt.title("Output")
plt.show()